<a class="anchor" id="0"></a>
# [Mechanisms of Action (MoA) Prediction](https://www.kaggle.com/c/lish-moa)

### I use the notebook [Pytorch CV|0.0145| LB| 0.01839 |](https://www.kaggle.com/riadalmadani/pytorch-cv-0-0145-lb-0-01839) from [riadalmadani](https://www.kaggle.com/riadalmadani) as a basis and will try to tune its various parameters. 

# Acknowledgements

* [MoA | Pytorch | 0.01859 | RankGauss | PCA | NN](https://www.kaggle.com/kushal1506/moa-pytorch-0-01859-rankgauss-pca-nn)
* [[MoA] Pytorch NN+PCA+RankGauss](https://www.kaggle.com/nayuts/moa-pytorch-nn-pca-rankgauss)
* [Pytorch CV|0.0145| LB| 0.01839 |](https://www.kaggle.com/riadalmadani/pytorch-cv-0-0145-lb-0-01839)
* [[New Baseline] Pytorch | MoA](https://www.kaggle.com/namanj27/new-baseline-pytorch-moa)
* [Deciding (n_components) in PCA](https://www.kaggle.com/kushal1506/deciding-n-components-in-pca)
* [Titanic - Featuretools (automatic FE&FS)](https://www.kaggle.com/vbmokin/titanic-featuretools-automatic-fe-fs)
* tuning and visualization from [Higher LB score by tuning mloss - upgrade & visual](https://www.kaggle.com/vbmokin/higher-lb-score-by-tuning-mloss-upgrade-visual)
* [Data Science for tabular data: Advanced Techniques](https://www.kaggle.com/vbmokin/data-science-for-tabular-data-advanced-techniques)

### My upgrade:

* PCA parameters
* Feature Selection methods
* Dropout
* Structuring of the notebook
* Tuning visualization
* Number of folds

I used the code from sources (please see above). But I am planning to develop this notebook. There are still promising areas for improvement and research of parameters.

<a class="anchor" id="0.1"></a>
## Table of Contents

1. [Import libraries](#1)
1. [My upgrade](#2)
    -  [Commit now](#2.1)
    -  [Previous commits](#2.2)
    -  [Parameters and LB score visualization](#2.3)
1. [Download data](#3)
1. [FE & Data Preprocessing](#4)
    - [RankGauss](#4.1)
    - [Seed](#4.2)    
    - [PCA features](#4.3)
    - [Feature selection](#4.4)
    - [CV folds](#4.5)
    - [Dataset Classes](#4.6)
    - [Smoothing](#4.7)
    - [Preprocessing](#4.8)
1. [Modeling](#5)
1. [Prediction & Submission](#6)

## 1. Import libraries<a class="anchor" id="1"></a>

[Back to Table of Contents](#0.1)

In [1]:
import sys
sys.path.append('../input/iterativestratification')

import numpy as np
import random
import pandas as pd
import os
import copy
import gc

import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.modules.loss import _WeightedLoss

import warnings
warnings.filterwarnings('ignore')

os.listdir('../input/lish-moa')

pd.set_option('max_columns', 2000)

## 2. My upgrade <a class="anchor" id="2"></a>

[Back to Table of Contents](#0.1)

### 2.1. Commit now <a class="anchor" id="2.1"></a>

[Back to Table of Contents](#0.1)

In [2]:
n_comp_GENES = 463
n_comp_CELLS = 60
VarianceThreshold_for_FS = 0.9
Dropout_Model = 0.25
QuantileTransformer_n_quantiles = 500 # instead of 100
print('n_comp_GENES', n_comp_GENES, 'n_comp_CELLS', n_comp_CELLS, 'total', n_comp_GENES + n_comp_CELLS)

n_comp_GENES 463 n_comp_CELLS 60 total 523


### 2.2 Previous commits <a class="anchor" id="2.2"></a>

[Back to Table of Contents](#0.1)

In [3]:
commits_df = pd.DataFrame(columns = ['n_commit', 'n_comp_GENES', 'n_comp_CELLS', 'train_features','VarianceThreshold_for_FS', 'Dropout_Model', 'LB_score', 'CV_logloss'])

### Commit 0 (parameters from https://www.kaggle.com/riadalmadani/pytorch-cv-0-0145-lb-0-01839, commit 8)

In [4]:
n=0
commits_df.loc[n, 'n_commit'] = 0                       # Number of commit
commits_df.loc[n, 'n_comp_GENES'] = 600                 # Number of output features for PCA for g-features
commits_df.loc[n, 'n_comp_CELLS'] = 50                  # Number of output features for PCA for c-features
commits_df.loc[n, 'VarianceThreshold_for_FS'] = 0.8     # Threshold for VarianceThreshold for feature selection
commits_df.loc[n, 'train_features'] = 1245              # Number features in the training dataframe after FE and before modeling
commits_df.loc[n, 'Dropout_Model'] = 0.2619422201258426 # Dropout in Model
commits_df.loc[n, 'CV_logloss'] = 0.01458269555140327   # Result CV logloss metrics
commits_df.loc[n, 'LB_score'] = 0.01839                 # LB score after submitting

### Commit 4

In [5]:
n=1
commits_df.loc[n, 'n_commit'] = 4
commits_df.loc[n, 'n_comp_GENES'] = 610
commits_df.loc[n, 'n_comp_CELLS'] = 55
commits_df.loc[n, 'VarianceThreshold_for_FS'] = 0.82
commits_df.loc[n, 'train_features'] = 1240
commits_df.loc[n, 'Dropout_Model'] = 0.25
commits_df.loc[n, 'CV_logloss'] =  0.014584545081734047
commits_df.loc[n, 'LB_score'] = 0.01839

### Commit 5

In [6]:
n=2
commits_df.loc[n, 'n_commit'] = 5
commits_df.loc[n, 'n_comp_GENES'] = 670
commits_df.loc[n, 'n_comp_CELLS'] = 67
commits_df.loc[n, 'VarianceThreshold_for_FS'] = 0.67
commits_df.loc[n, 'train_features'] = 1298
commits_df.loc[n, 'Dropout_Model'] = 0.25
commits_df.loc[n, 'CV_logloss'] =  0.014588561242139069
commits_df.loc[n, 'LB_score'] = 0.01840

### Commit 6

In [7]:
n=3
commits_df.loc[n, 'n_commit'] = 6
commits_df.loc[n, 'n_comp_GENES'] = 450
commits_df.loc[n, 'n_comp_CELLS'] = 45
commits_df.loc[n, 'VarianceThreshold_for_FS'] = 0.67
commits_df.loc[n, 'train_features'] = 1297
commits_df.loc[n, 'Dropout_Model'] = 0.25
commits_df.loc[n, 'CV_logloss'] =  0.014586229676302227
commits_df.loc[n, 'LB_score'] = 0.01840

### Commit 9

In [8]:
n=4
commits_df.loc[n, 'n_commit'] = 9
commits_df.loc[n, 'n_comp_GENES'] = 463
commits_df.loc[n, 'n_comp_CELLS'] = 60
commits_df.loc[n, 'VarianceThreshold_for_FS'] = 0.9
commits_df.loc[n, 'train_features'] = 1219
commits_df.loc[n, 'Dropout_Model'] = 0.25
commits_df.loc[n, 'CV_logloss'] =  0.014572358066092783
commits_df.loc[n, 'LB_score'] = 0.01839

### Commit 10

In [9]:
n=5
commits_df.loc[n, 'n_commit'] = 10
commits_df.loc[n, 'n_comp_GENES'] = 463
commits_df.loc[n, 'n_comp_CELLS'] = 80
commits_df.loc[n, 'VarianceThreshold_for_FS'] = 0.92
commits_df.loc[n, 'train_features'] = 1214
commits_df.loc[n, 'Dropout_Model'] = 0.25
commits_df.loc[n, 'CV_logloss'] =  0.014571552074579226
commits_df.loc[n, 'LB_score'] = 0.01841

### Commit 12

In [10]:
n=6
commits_df.loc[n, 'n_commit'] = 12
commits_df.loc[n, 'n_comp_GENES'] = 450
commits_df.loc[n, 'n_comp_CELLS'] = 65
commits_df.loc[n, 'VarianceThreshold_for_FS'] = 0.9
commits_df.loc[n, 'train_features'] = 1219
commits_df.loc[n, 'Dropout_Model'] = 0.25
commits_df.loc[n, 'CV_logloss'] = 0.01458043214513875
commits_df.loc[n, 'LB_score'] = 0.01840

### Commit 13

In [11]:
n=7
commits_df.loc[n, 'n_commit'] = 13
commits_df.loc[n, 'n_comp_GENES'] = 463
commits_df.loc[n, 'n_comp_CELLS'] = 60
commits_df.loc[n, 'VarianceThreshold_for_FS'] = 0.9
commits_df.loc[n, 'train_features'] = 1219
commits_df.loc[n, 'Dropout_Model'] = 0.4
commits_df.loc[n, 'CV_logloss'] = 0.014625250378417162
commits_df.loc[n, 'LB_score'] = 0.01844

### Commit 14

In [12]:
n=8
commits_df.loc[n, 'n_commit'] = 14
commits_df.loc[n, 'n_comp_GENES'] = 463
commits_df.loc[n, 'n_comp_CELLS'] = 60
commits_df.loc[n, 'VarianceThreshold_for_FS'] = 0.01
commits_df.loc[n, 'train_features'] = 1604
commits_df.loc[n, 'Dropout_Model'] = 0.25
commits_df.loc[n, 'CV_logloss'] = 0.014713482787703418
commits_df.loc[n, 'LB_score'] = 0.01849

### Commit 18

In [13]:
n=9
commits_df.loc[n, 'n_commit'] = 18
commits_df.loc[n, 'n_comp_GENES'] = 363
commits_df.loc[n, 'n_comp_CELLS'] = 60
commits_df.loc[n, 'VarianceThreshold_for_FS'] = 0.9
commits_df.loc[n, 'train_features'] = 1219
commits_df.loc[n, 'Dropout_Model'] = 0.25
commits_df.loc[n, 'CV_logloss'] = 0.014568689235607534
commits_df.loc[n, 'LB_score'] = 0.01841

### Commit 19

In [14]:
n=10
commits_df.loc[n, 'n_commit'] = 19
commits_df.loc[n, 'n_comp_GENES'] = 550
commits_df.loc[n, 'n_comp_CELLS'] = 60
commits_df.loc[n, 'VarianceThreshold_for_FS'] = 0.91
commits_df.loc[n, 'train_features'] = 1218
commits_df.loc[n, 'Dropout_Model'] = 0.25
commits_df.loc[n, 'CV_logloss'] = 0.014577509066710863
commits_df.loc[n, 'LB_score'] = 0.01841

### Commit 20

In [15]:
n=11
commits_df.loc[n, 'n_commit'] = 20
commits_df.loc[n, 'n_comp_GENES'] = 463
commits_df.loc[n, 'n_comp_CELLS'] = 60
commits_df.loc[n, 'VarianceThreshold_for_FS'] = 0.9
commits_df.loc[n, 'train_features'] = 1218
commits_df.loc[n, 'Dropout_Model'] = 0.25
commits_df.loc[n, 'CV_logloss'] = 0.014572358066092783
commits_df.loc[n, 'LB_score'] = 0.01839

### 2.3 Parameters and LB score visualization <a class="anchor" id="2.3"></a>

[Back to Table of Contents](#0.1)

In [16]:
commits_df['n_comp_total'] = commits_df['n_comp_GENES'] + commits_df['n_comp_CELLS']
commits_df['seed'] = 42

In [17]:
commits_df['l_rate'] = 1e-3
commits_df.loc[11, 'l_rate'] = 5e-4

In [18]:
# Find and mark minimun value of LB score
commits_df['LB_score'] = pd.to_numeric(commits_df['LB_score'])
commits_df = commits_df.sort_values(by=['LB_score', 'CV_logloss'], ascending = True).reset_index(drop=True)
commits_df['min'] = 0
commits_df.loc[0, 'min'] = 1
commits_df

,n_commit,n_comp_GENES,n_comp_CELLS,train_features,VarianceThreshold_for_FS,Dropout_Model,LB_score,CV_logloss,n_comp_total,seed,l_rate,min
0,9,463,60,1219,0.9,0.25,0.01839,0.0145724,523,42,0.0010,1
1,20,463,60,1218,0.9,0.25,0.01839,0.0145724,523,42,0.0005,0
2,0,600,50,1245,0.8,0.261942,0.01839,0.0145827,650,42,0.0010,0
3,4,610,55,1240,0.82,0.25,0.01839,0.0145845,665,42,0.0010,0
4,12,450,65,1219,0.9,0.25,0.01840,0.0145804,515,42,0.0010,0
5,6,450,45,1297,0.67,0.25,0.01840,0.0145862,495,42,0.0010,0
6,5,670,67,1298,0.67,0.25,0.01840,0.0145886,737,42,0.0010,0
7,18,363,60,1219,0.9,0.25,0.01841,0.0145687,423,42,0.0010,0
8,10,463,80,1214,0.92,0.25,0.01841,0.0145716,543,42,0.0010,0
9,19,550,60,1218,0.91,0.25,0.01841,0.0145775,610,42,0.0010,0


In [19]:
commits_df.sort_values(by=['CV_logloss'], ascending = True)

,n_commit,n_comp_GENES,n_comp_CELLS,train_features,VarianceThreshold_for_FS,Dropout_Model,LB_score,CV_logloss,n_comp_total,seed,l_rate,min
7,18,363,60,1219,0.9,0.25,0.01841,0.0145687,423,42,0.0010,0
8,10,463,80,1214,0.92,0.25,0.01841,0.0145716,543,42,0.0010,0
0,9,463,60,1219,0.9,0.25,0.01839,0.0145724,523,42,0.0010,1
1,20,463,60,1218,0.9,0.25,0.01839,0.0145724,523,42,0.0005,0
9,19,550,60,1218,0.91,0.25,0.01841,0.0145775,610,42,0.0010,0
4,12,450,65,1219,0.9,0.25,0.01840,0.0145804,515,42,0.0010,0
2,0,600,50,1245,0.8,0.261942,0.01839,0.0145827,650,42,0.0010,0
3,4,610,55,1240,0.82,0.25,0.01839,0.0145845,665,42,0.0010,0
5,6,450,45,1297,0.67,0.25,0.01840,0.0145862,495,42,0.0010,0
6,5,670,67,1298,0.67,0.25,0.01840,0.0145886,737,42,0.0010,0


In [20]:
# Interactive plot with results of parameters tuning
fig = px.scatter_3d(commits_df, x='n_comp_GENES', y='n_comp_CELLS', z='LB_score', color = 'min', 
                    symbol = 'Dropout_Model',
                    title='Parameters and LB score visualization of MoA solutions')
fig.update(layout=dict(title=dict(x=0.1)))

In [21]:
# Interactive plot with results of parameters tuning
fig = px.scatter_3d(commits_df, x='train_features', y='VarianceThreshold_for_FS', z='LB_score', color = 'min', 
                    symbol = 'l_rate',
                    title='Parameters and LB score visualization of MoA solutions')
fig.update(layout=dict(title=dict(x=0.1)))

In [22]:
# Interactive plot with results of parameters tuning
fig = px.scatter_3d(commits_df, x='train_features', y='CV_logloss', z='LB_score', color = 'min', 
                    symbol = 'l_rate',
                    title='Parameters and LB score visualization of MoA solutions')
fig.update(layout=dict(title=dict(x=0.1)))

In [23]:
# Interactive plot with results of parameters tuning
commits_df_1841 = commits_df[commits_df.LB_score <= 0.01841]
fig = px.scatter_3d(commits_df_1841, x='train_features', y='CV_logloss', z='LB_score', color = 'min', 
                    symbol = 'l_rate',
                    title='Parameters and LB score visualization of MoA solutions')
fig.update(layout=dict(title=dict(x=0.1)))

As already noted, **LEARNING_RATE** is adaptively adjusted. Therefore, it makes no sense to tune it - the result is the same.

### It is recommended:
* **n_comp_GENES** smaller, 
* **n_comp_CELLS** more,
* **VarianceThreshold_for_FS** more, so that **train_features** is less.

## 3. Download data<a class="anchor" id="3"></a>

[Back to Table of Contents](#0.1)

In [24]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

## 4. FE & Data Preprocessing <a class="anchor" id="4"></a>

[Back to Table of Contents](#0.1)

In [25]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

### 4.1 RankGauss<a class="anchor" id="4.1"></a>

[Back to Table of Contents](#0.1)

In [26]:
# RankGauss - transform to Gauss

for col in (GENES + CELLS):

    transformer = QuantileTransformer(n_quantiles=QuantileTransformer_n_quantiles,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

### 4.2 Seed<a class="anchor" id="4.2"></a>

[Back to Table of Contents](#0.1)

In [27]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

### 4.3 PCA features<a class="anchor" id="4.3"></a>

[Back to Table of Contents](#0.1)

In [28]:
len(GENES)

772

In [29]:
# GENES

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp_GENES, random_state=42).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp_GENES)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp_GENES)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [30]:
len(CELLS)

100

In [31]:
# CELLS

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp_CELLS, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp_CELLS)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp_CELLS)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [32]:
train_features.shape

(23814, 1399)

In [33]:
train_features.head(5)

sig_id cp_type  cp_time cp_dose       g-0       g-1       g-2  \
0  id_000644bb2  trt_cp       24      D1  1.135329  0.907871 -0.416056   
1  id_000779bfc  trt_cp       72      D1  0.119307  0.682082  0.272066   
2  id_000a6266a  trt_cp       48      D1  0.779911  0.946241  1.425076   
3  id_0015fd391  trt_cp       48      D1 -0.735068 -0.274041 -0.437981   
4  id_001626bd3  trt_cp       72      D2 -0.452153 -0.476994  0.972847   

        g-3       g-4       g-5       g-6       g-7       g-8       g-9  \
0 -0.968039 -0.255340 -1.015601 -1.366836 -0.025365  0.679176 -0.309849   
1  0.080090  1.204343  0.686856  0.314253  0.555638 -0.537128  0.838334   
2 -0.131449 -0.006720  1.494332  0.234189  0.365066 -0.005485  1.240305   
3  0.758995  2.450858 -0.858918 -2.302688  0.308319 -0.192281 -1.386051   
4  0.971264  1.464223 -0.870248 -0.375927 -0.204570 -1.064268  0.849390   

       g-10      g-11      g-12      g-13      g-14      g-15      g-16  \
0  1.525286  0.170935  0.622365 -0.562485  0.280808 -1.057699 -1.141974   
1 -1.246925 -0.576240 -0.211084  0.545683  0.149631  0.390751  0.399016   
2 -0.642049 -0.770758 -0.130254 -2.318567  0.878104 -0.546810  0.471513   
3 -1.001516 -0.496024 -1.147069 -0.817017 -1.766285  1.393524 -0.228769   
4 -0.335974 -0.704161 -1.246515  1.183070  0.318119  0.342736 -0.025681   

       g-17      g-18      g-19      g-20      g-21      g-22      g-23  \
0  0.870306  0.380826 -0.510288 -0.723759 -1.330358 -1.677516  1.250091   
1  0.398856 -0.840190  0.499081  1.227475 -0.135705 -0.397160 -0.440854   
2 -0.406609 -0.306381 -0.042828 -0.105768 -1.010887 -1.966401  0.533838   
3 -1.053845  0.245322 -2.143516  2.100103 -1.350884 -1.101454 -1.060164   
4 -1.567247 -0.852875 -0.704816 -0.447090 -0.291741  1.054528  0.489594   

       g-24      g-25      g-26      g-27      g-28      g-29      g-30  \
0  0.559176  0.401912  0.240643  0.165332 -0.522229  0.771816  0.416076   
1  0.396454 -0.583305  0.632018  0.220515 -0.744947 -0.187210 -0.120986   
2  0.626802  0.698398 -1.379040  2.684515 -0.094260  1.580265 -1.458723   
3 -1.422587  0.002020 -1.052717  0.262879 -0.457188 -1.247654 -0.977581   
4  1.690332  1.002605  1.072273 -0.553269 -1.541124  0.377033 -0.154322   

       g-31      g-32      g-33      g-34      g-35      g-36      g-37  \
0 -1.101826 -0.052577 -0.441763 -0.199609  0.274327  0.384496 -0.585163   
1 -0.553801  1.661572 -0.369647  0.352018 -0.284531  0.202957  0.782723   
2  0.812197  0.736989  0.205093 -0.001970 -1.675028  0.148218 -0.154667   
3  0.124649 -2.089931  1.037023  0.779339 -2.174276 -1.410319 -1.057475   
4  0.843148  1.135636 -0.138614  1.489816 -1.024878 -1.320718  1.632353   

       g-38      g-39      g-40      g-41      g-42      g-43      g-44  \
0 -0.739915 -0.370729 -0.010726 -0.344069 -1.360411  0.422363 -1.256823   
1 -0.358209 -0.090183  0.206734 -0.524834 -1.341580  0.616242  1.869396   
2  1.705509  1.308575  0.546316 -0.721698  0.100203 -0.533278 -0.576574   
3  1.416654 -0.641283 -0.886928 -0.003989  0.246237 -2.035029 -0.924722   
4  0.772176 -0.152770  0.221993  0.166559  1.428742  0.277949  0.845983   

       g-45      g-46      g-47      g-48      g-49      g-50      g-51  \
0  0.330448 -0.348114 -0.021747  0.060276  0.698781 -0.180929 -0.660585   
1  1.335859  0.557251 -0.500348  0.234926 -1.097220  1.794077 -0.675346   
2 -0.784372  0.246092 -1.056173 -0.666818  0.741509 -0.148595 -0.350092   
3 -0.993141 -0.418046  0.124363 -0.647933  1.022917 -1.320744 -0.810764   
4  1.393001  1.477321 -1.273638 -0.495634 -0.996965 -0.124764 -0.511316   

       g-52      g-53      g-54      g-55      g-56      g-57      g-58  \
0  0.841582  0.567010 -0.153002 -0.228603 -0.768572  0.278112  0.798036   
1 -0.348146 -1.004637 -0.773271 -0.172292 -0.731966  0.581736  0.435969   
2  0.833041  0.321507 -1.086167 -0.603463  1.368715  0.059812 -0.258912   
3 -1.429021  1.712910 -0.934218 -1.358013 -0.692150  0.528680  0.091848   
4  0.949564 -0.027632  0.149770 

### 4.4 Feature selection<a class="anchor" id="4.4"></a>

[Back to Table of Contents](#0.1)

In [34]:
data = train_features.append(test_features)
data

sig_id cp_type  cp_time cp_dose       g-0       g-1       g-2  \
0     id_000644bb2  trt_cp       24      D1  1.135329  0.907871 -0.416056   
1     id_000779bfc  trt_cp       72      D1  0.119307  0.682082  0.272066   
2     id_000a6266a  trt_cp       48      D1  0.779911  0.946241  1.425076   
3     id_0015fd391  trt_cp       48      D1 -0.735068 -0.274041 -0.437981   
4     id_001626bd3  trt_cp       72      D2 -0.452153 -0.476994  0.972847   
...            ...     ...      ...     ...       ...       ...       ...   
3977  id_ff7004b87  trt_cp       24      D1  0.599228 -0.690841  2.410293   
3978  id_ff925dd0d  trt_cp       24      D1 -0.844647 -0.282627  2.017895   
3979  id_ffb710450  trt_cp       72      D1 -0.564750 -0.146369  0.233767   
3980  id_ffbb869f2  trt_cp       48      D2 -1.598704 -1.932100 -0.617743   
3981  id_ffd5800b6  trt_cp       72      D1 -0.740343  0.510587  0.308094   

           g-3       g-4       g-5       g-6       g-7       g-8       g-9  \
0    -0.968039 -0.255340 -1.015601 -1.366836 -0.025365  0.679176 -0.309849   
1     0.080090  1.204343  0.686856  0.314253  0.555638 -0.537128  0.838334   
2    -0.131449 -0.006720  1.494332  0.234189  0.365066 -0.005485  1.240305   
3     0.758995  2.450858 -0.858918 -2.302688  0.308319 -0.192281 -1.386051   
4     0.971264  1.464223 -0.870248 -0.375927 -0.204570 -1.064268  0.849390   
...        ...       ...       ...       ...       ...       ...       ...   
3977 -0.966991  1.073563  1.786479  0.093157 -0.777859 -1.302970  0.710507   
3978  0.507095  0.641807  0.434798 -1.008719  0.005065  0.316838  1.884350   
3979 -1.052821  0.066455  0.635301 -0.019257  1.183612  0.028173  1.396210   
3980  1.321004 -1.620956 -0.264244 -0.487322 -1.280867  1.176414  0.648341   
3981 -0.079867 -0.080527  1.098847 -1.177453  0.635053 -0.078526  0.241759   

          g-10      g-11      g-12      g-13      g-14      g-15      g-16  \
0     1.525286  0.170935  0.622365 -0.562485  0.280808 -1.057699 -1.141974   
1    -1.246925 -0.576240 -0.211084  0.545683  0.149631  0.390751  0.399016   
2    -0.642049 -0.770758 -0.130254 -2.318567  0.878104 -0.546810  0.471513   
3    -1.001516 -0.496024 -1.147069 -0.817017 -1.766285  1.393524 -0.228769   
4    -0.335974 -0.704161 -1.246515  1.183070  0.318119  0.342736 -0.025681   
...        ...       ...       ...       ...       ...       ...       ...   
3977  0.323337  0.953354  0.847746 -0.961399 -0.802193 -1.728311 -0.023723   
3978  0.222828  0.468210  0.943055 -1.360951  0.275127  2.272043  1.311850   
3979  0.578947 -0.586697 -0.242103 -0.387376  0.648839 -1.256767 -0.401755   
3980  1.863316  0.778689  1.243290  1.314189  1.392839  0.824837 -0.387557   
3981  0.270831  0.288072 -0.273702  1.371577 -0.648578  1.319761  0.494015   

          g-17      g-18      g-19      g-20      g-21      g-22      g-23  \
0     0.870306  0.380826 -0.510288 -0.723759 -1.330358 -1.677516  1.250091   
1     0.398856 -0.840190  0.499081  1.227475 -0.135705 -0.397160 -0.440854   
2    -0.406609 -0.306381 -0.042828 -0.105768 -1.010887 -1.966401  0.533838   
3    -1.053845  0.245322 -2.143516  2.100103 -1.350884 -1.101454 -1.060164   
4    -1.567247 -0.852875 -0.704816 -0.447090 -0.291741  1.054528  0.489594   
...        ...       ...       ...       ...       ...       ...       ...   
3977  0.382146  0.792974 -1.423784  1.281016 -0.953311  0.970004  1.505775   
3978 -0.248137  0.757823  0.885465  0.838818 -0.930381 -0.250864  0.691935   
3979  1.316144  0.963133 -1.226402 -1.346128 -0.164968 -0.312849  0.691935   
3980  0.708978  0.029391 -0.121104  1.221609 -1.790924  0.328457 -1.036820   
3981  0.588086 -1.151503 -1.862807  0.044387  0.464618 -1.265608  0.108212   

          g-24      g-25      g-26      g-27      g-28      g-29      g-30  \
0     0.559176  0.401912  0.240643  0.165332 -0.522229  0.771816  0.416076   
1     0.396454 -0.583305  0.632018  0.220515 -0.744947 -0.187210 -0.120986   
2     0.626802  0.698398 -1.379040  2.6845

In [35]:
var_thresh = VarianceThreshold(VarianceThreshold_for_FS)
data = train_features.append(test_features)
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features.shape

(23814, 1019)

In [36]:
train_features.head(5)

,sig_id,cp_type,cp_time,cp_dose,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014
0,i

In [37]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_scored.columns]

In [38]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [39]:
train.head(5)

,sig_id,cp_time,cp_dose,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,5-alpha_red

In [40]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

### 4.5 CV folds<a class="anchor" id="4.5"></a>

[Back to Table of Contents](#0.1)

In [41]:
folds = train.copy()

mskf = MultilabelStratifiedKFold(n_splits=7)

for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
    folds.loc[v_idx, 'kfold'] = int(f)

folds['kfold'] = folds['kfold'].astype(int)
folds

sig_id cp_time cp_dose         0         1         2         3  \
0      id_000644bb2      24      D1  1.135329  0.907871 -0.416056 -0.968039   
1      id_000779bfc      72      D1  0.119307  0.682082  0.272066  0.080090   
2      id_000a6266a      48      D1  0.779911  0.946241  1.425076 -0.131449   
3      id_0015fd391      48      D1 -0.735068 -0.274041 -0.437981  0.758995   
4      id_001626bd3      72      D2 -0.452153 -0.476994  0.972847  0.971264   
...             ...     ...     ...       ...       ...       ...       ...   
21943  id_fff8c2444      72      D1  0.237498 -1.230100  0.219031 -0.366167   
21944  id_fffb1ceed      24      D2  0.208495 -0.022136 -0.236379 -0.797301   
21945  id_fffb70c0c      24      D2 -1.910311  0.586229 -0.588718  1.296786   
21946  id_fffcb9e7c      24      D1  0.816563  0.417084  0.431959  0.300768   
21947  id_ffffdd77b      72      D1 -1.244018  1.569378 -0.268627  1.085885   

              4         5         6         7         8         9        10  \
0     -0.255340 -1.015601 -1.366836 -0.025365  0.679176 -0.309849  1.525286   
1      1.204343  0.686856  0.314253  0.555638 -0.537128  0.838334 -1.246925   
2     -0.006720  1.494332  0.234189  0.365066 -0.005485  1.240305 -0.642049   
3      2.450858 -0.858918 -2.302688  0.308319 -0.192281 -1.386051 -1.001516   
4      1.464223 -0.870248 -0.375927 -0.204570 -1.064268  0.849390 -0.335974   
...         ...       ...       ...       ...       ...       ...       ...   
21943 -0.329890  0.569039 -0.151299 -0.249377 -0.940954 -0.030719  0.703196   
21944 -0.675392  0.917995  0.737173  0.507993 -0.623786 -0.041841  0.293054   
21945 -1.003709  0.850604 -0.304687 -0.704853  0.629444  0.645803 -0.292201   
21946  1.071485 -0.023108  0.049803  0.099449  0.530589 -1.193376  0.734723   
21947 -0.511611 -2.108523 -1.629218  1.429255  2.282361  1.254978  2.104993   

             11        12        13        14        15        16        17  \
0      0.170935  0.622365 -0.562485  0.280808 -1.057699 -1.141974  0.870306   
1     -0.576240 -0.211084  0.545683  0.149631  0.390751  0.399016  0.398856   
2     -0.770758 -0.130254 -2.318567  0.878104 -0.546810  0.471513 -0.406609   
3     -0.496024 -1.147069 -0.817017 -1.766285  1.393524 -0.228769 -1.053845   
4     -0.704161 -1.246515  1.183070  0.318119  0.342736 -0.025681 -1.567247   
...         ...       ...       ...       ...       ...       ...       ...   
21943 -0.698886  0.447134  0.138846  1.110891 -0.313375  0.108114  0.277147   
21944  0.196910 -0.682243 -1.509573  0.379643 -0.763011  0.227242  0.227157   
21945  1.780664 -0.364732 -0.861345  1.167375  0.967810 -0.497108  0.184342   
21946  0.157676  1.704163  0.246641  0.783632 -2.244750 -0.004669  0.836996   
21947  1.286898  1.891905  1.620959 -0.752249  1.191279 -1.381115  0.031180   

             18        19        20        21        22        23        24  \
0      0.380826 -0.510288 -0.723759 -1.330358 -1.677516  1.250091  0.559176   
1     -0.840190  0.499081  1.227475 -0.135705 -0.397160 -0.440854  0.396454   
2     -0.306381 -0.042828 -0.105768 -1.010887 -1.966401  0.533838  0.626802   
3      0.245322 -2.143516  2.100103 -1.350884 -1.101454 -1.060164 -1.422587   
4     -0.852875 -0.704816 -0.447090 -0.291741  1.054528  0.489594  1.690332   
...         ...       ...       ...       ...       ...       ...       ...   
21943  0.053524  0.231275 -0.650470 -0.748782  0.739493 -0.691935  0.191677   
21944 -1.512780 -0.125211  1.124498 -0.508261  0.825976  0.534224 -0.699562   
21945  0.030898  0.189699 -0.291986 -1.120138  0.140274 -1.824105 -0.980705   
21946  0.433803  1.470173 -0.962249  1.380819  0.548125  2.920571  0.629901   
21947 -1.769763 -0.069813 -0.255239 -0.963117 -1.870427 -1.308144 -1.208618   

             25        26        27        28        29        30        31  \
0      0.401912  0.240643  0.165332 -0.522229  0.771816  0.416076 -1.101826   
1     -0.583305  0.632018  0.220515 -0.744947 -0.187210 -0.

In [42]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target.shape)
print(sample_submission.shape)

(21948, 1224)
(21948, 1225)
(3624, 1018)
(21948, 207)
(3982, 207)


### 4.6 Dataset Classes<a class="anchor" id="4.6"></a>

[Back to Table of Contents](#0.1)

In [43]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct    

In [44]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

### 4.7 Smoothing<a class="anchor" id="4.7"></a>

[Back to Table of Contents](#0.1)

In [45]:
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

### 4.8 Preprocessing<a class="anchor" id="4.8"></a>

[Back to Table of Contents](#0.1)

In [46]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [47]:
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]
len(feature_cols)

1020

## 5. Modeling<a class="anchor" id="5"></a>

[Back to Table of Contents](#0.1)

In [48]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 7
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=1500

In [49]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(Dropout_Model)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(Dropout_Model)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x
    
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))    

In [50]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
   
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    
    model.load_state_dict(torch.load(f"FOLD{fold}_.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions

## 6. Prediction & Submission <a class="anchor" id="6"></a>

[Back to Table of Contents](#0.1)

In [51]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [52]:
# Averaging on multiple SEEDS

SEED = [0, 1, 2, 3, 4, 5, 6]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions


FOLD: 0, EPOCH: 0, train_loss: 0.47342149868962313
FOLD: 0, EPOCH: 0, valid_loss: 0.025350222438573836
FOLD: 0, EPOCH: 1, train_loss: 0.023868009131275066
FOLD: 0, EPOCH: 1, valid_loss: 0.019066891223192214
FOLD: 0, EPOCH: 2, train_loss: 0.02188564185686663
FOLD: 0, EPOCH: 2, valid_loss: 0.017808807864785194
FOLD: 0, EPOCH: 3, train_loss: 0.020712724158248933
FOLD: 0, EPOCH: 3, valid_loss: 0.017629054896533488
FOLD: 0, EPOCH: 4, train_loss: 0.020368740431406872
FOLD: 0, EPOCH: 4, valid_loss: 0.017397941425442695
FOLD: 0, EPOCH: 5, train_loss: 0.020247735349195346
FOLD: 0, EPOCH: 5, valid_loss: 0.01749847948551178
FOLD: 0, EPOCH: 6, train_loss: 0.020309730477276303
FOLD: 0, EPOCH: 6, valid_loss: 0.017511921003460884
FOLD: 0, EPOCH: 7, train_loss: 0.020320960415565237
FOLD: 0, EPOCH: 7, valid_loss: 0.01762143298983574
FOLD: 0, EPOCH: 8, train_loss: 0.020328018712956888
FOLD: 0, EPOCH: 8, valid_loss: 0.017474880442023277
FOLD: 0, EPOCH: 9, train_loss: 0.020292326427844107
FOLD: 0, EPOCH: 

In [53]:
train_targets_scored

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [54]:
len(target_cols)

206

In [55]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)    

CV log_loss:  0.014572447523411875


In [56]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('submission.csv', index=False)

In [57]:
sub.shape

(3982, 207)

[Go to Top](#0)